In [ ]:
using RigidBodySim
using RigidBodyDynamics
using Plots
using LinearAlgebra
using Statistics
using SpecialFunctions
using Swarm
using Trajectory
using PerformanceIndex
include("../src/fracionario.jl")
include("../src/simulation.jl");

In [ ]:
urdf = "../src/pelican.urdf"
robot = parse_urdf(urdf) 
initial_state = MechanismState(robot) # cria condições iniciais nulas para o sistema 
num_joints = num_positions(initial_state); # quantidade total de juntas

In [ ]:
time_end = 2. #tempo total de simulação
Δt = 0.05  #salva a cada Δt segundos
lb = vcat(fill(10., num_joints * 2), zeros(2) )# limite inferior dos ganhos para otimização
                           # e do expoente da derivada
ub = fill( 10000. ,num_joints * 2) #limite superior dos ganhos para otimização
ub = vcat(ub, ones(num_joints))
saturation_value = [200., 15.];

In [ ]:
# Dados para dinâmica inversa e geração das trajetórias

final_position = 0.8
xr, vr, ar, jr = minimumjerk(fill(final_position , num_joints), time_end);

In [ ]:
function objetivo(data::Vector{Float64})
    kp_local = data[1:num_joints]
    kv_local = data[(num_joints + 1):(num_joints * 2)]
    lambda_local = [1.,1.]
    
    ex, ev, ea, ej, tout, ta, tj = erroPDDigitalFractional(kp_local, 
        kv_local, lambda_local, xr, vr, ar, jr, robot,
        initial_state, Δt, time_end, maxTorque = saturation_value);
    return sum(iae(ej)) + sum(iae(ex))
end;

In [ ]:
#variáveis
ψ  = 0.7  # => fator de constrição
ϕp = 0.5  # => coeficiente de aceleração partícula
ϕg = 0.5  # => coeficiente de aceleração grupo
iter = 150  # => iterações
num_particles = 50  # => quantidade de partículas 
min_step = 1e-4 # => critério de parada
min_func = 1e-4 # => critério de parada
log = false     # => exibe mensagens de log
 
#gera partículas iguais para cada otimização
particles_pso = Particles(num_particles, lb, ub);

In [ ]:
pso(particles_pso, objetivo, verbose = log, minfunc = min_func,
    minstep = min_step, psi = ψ, phip = ϕp , phig = ϕg,
    maxiter = iter, localsearch = false, neighborhood = 10)

In [ ]:
println("kp = $(particles_pso.best_position[1:num_joints])")
println("kv = $(particles_pso.best_position[(num_joints+1):(num_joints * 2)])")
println("Fitness = $(particles_pso.best_value)")

In [ ]:
plot(particles_pso.history, xlabel = "Iteration", ylabel="Fitness")